In [ ]:
import dill

In [ ]:
dill.dump_session('notebook_env.db')

In [ ]:
dill.load_session('notebook_env.db')

# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment (we use Python 3.6.8) there if you'd like: 
```
$ cd .../<project-directory>
$ python -m virtualenv env 
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you:
```
$ python 
>>> import cosmicfish as cf
>>> cf.install_class('<project-directory>/class')
>>> exit()
```

# Import & Configure _cosmicfish_

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
#Import relevant packages
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/datastore/")
classpath = os.path.join(projectdir, "class")

Specify the granularity of numerical derivatives.

In [ ]:
derivative_step = 0.005 #How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.01 #For calculating numerical integral wrt mu between -1 and 1 

# Specify Example Fiducial Cosmology 

Here we specify the fiducial cosmology used in our forecasts. For the case of the degenerate neutrino forecast, we specify the mass of single neutrino. 

In [ ]:
#Setup fiducial cosmology
neutrinofid = {
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        "T_ncdm" : 1.95/2.726, 
        "m_ncdm" : 0.03, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : None #Not used for neutrino forecasts
        } 


For light relic forecasts, we choose whether to vary with respect to the relic mass or the relic temperature, specified by the "relic_fix" parameter.

In [ ]:
# For matching the neutrino/relic forecasts, we must modify temp/mass according to equation 44 
# of the Cora/Julian paper. That is: 
# m_chi = m_nu * 3^(1/4) ***where m_nu is the mass of a single neutrino in the degenerate hierarchy
# T_chi = 1.95 * 3^(1/4)

relicfid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.95 * np.power(3., 1./4.))/2.726, # Units [T_cmb]  Are Julian's matrices at 1.64 K? 
        "m_ncdm" : 0.02 * np.power(3., 1./4.), # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046 - (3 * 1.0132), 
        "relic_fix" : "T_ncdm" # Fix T_ncdm or M_ncdm 
        }

relicfid2 = dict(relicfid, **{"relic_fix" : "m_ncdm"})

# Specify Experiment Observational Parameters

The experimental redshift configuration and noise is specified here. 

In [ ]:
#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence 

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `convergence` class of `cosmicfish`. 

We pass to `convergence` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology by step sizes specified by the user and compute the corresponding power spectrum derivatives. 

All 'varyfactors' are computed relative to the fiducial cosmology. That is:

dtheta = varyfactor * theta_fiducial

**WARNING: This process takes a considerable amount of time (~5 mins per varyfactor w/o prexisting data). Only run when you need to.**

In [ ]:
neutrinoconvergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
    neutrinofid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    mu_step=mu_integral_step,
    varyfactors=[0.003, 0.005, 0.007] # Relative factors used to compute convergence
    )
neutrinoconvergencetest.gen_all_plots() # Display convergence plots

In [ ]:
relicconvergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'relic', # 'relic' or 'neutrino' forecasting scheme 
    relicfid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    mu_step=mu_integral_step,
    varyfactors=[0.003, 0.005, 0.007] # Relative factors used to compute convergence
    )
relicconvergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecast

In [ ]:
neutrinoforecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    neutrinofid, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

neutrinoforecast.gen_pm()
neutrinoforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        #'M_ncdm',                                    
        'omega_ncdm',                                 
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'],
    mu_step=mu_integral_step, 
    skipgen=False)

In [ ]:
relicforecast = cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    relicfid, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

relicforecast.gen_pm()
relicforecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        #'M_ncdm',                                    
        'omega_ncdm',
        #'T_ncdm',
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'],
    mu_step=mu_integral_step, 
    skipgen=False)

In [ ]:
relicforecast2 = cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    relicfid2, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

relicforecast2.gen_pm()
relicforecast2.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        #'M_ncdm',                                    
        'omega_ncdm',
        #'T_ncdm',
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'],
    mu_step=mu_integral_step, 
    skipgen=False)

### Add CMB Data

Add CMB prior information to forecast. 

In [ ]:
neutrinoforecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm'],
    fisherpath=os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

In [ ]:
relicforecast.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm'],
    os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

relicforecast2.load_cmb_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                                                             
        'omega_ncdm'],
    os.path.join(cf.priors_directory(), "CMBS4_Fisher_Neutrinos.dat"))

### Export Fisher Matrices

Export the CMB, LSS, and combined covariance matrices. 

In [ ]:
neutrinoforecast.export_matrices("~/Desktop")

In [ ]:
pd.DataFrame(neutrinoforecast.pandas_full_fisher)

In [ ]:
relicforecast.export_matrices("~/Desktop")
relicforecast2.export_matrices("~/Desktop")

# Light Relic/Degenerate Neutrino Validation Plot

We now want to see how the error in non-cold dark matter abundance varies with fiducial mass.  To do this, we will generate a forecast at each fiducial mass of interest. We would like to ensure that the light relic forecast produces identical results as an equivalent degenerate neutrino forecast. To do this, we will: 

(1) Set T_relic = 3^(1/4) T_neutrino = 3^(1/4) * 1.95 [K]

(2) Set m_relic = 3^(1/4) m_neutrino

(3) In both cases, we will reduce the LCDM value of N_eff (3.046) by 1.0132 for each neutrino, and three times this value for the case of the relic. 

In [ ]:
relicmasses = np.geomspace(0.1, 10.0, 21) * (1./3.) * np.power(3., 1./4.)
relicfiducialset = [dict(relicfid, **{'m_ncdm': relicmasses[midx]}) for midx, mval in enumerate(relicmasses)]

neutrinomasses = np.geomspace(0.1, 10.0, 21) * (1./3.)
neutrinofiducialset = [dict(neutrinofid, **{'m_ncdm': neutrinomasses[midx]}) 
                       for midx, mval in enumerate(neutrinomasses)]

In [ ]:
relicforecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(relicfiducialset)]

neutrinoforecastset = [cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    fidval, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(neutrinofiducialset)]

In [ ]:
for fcst in relicforecastset: 
    fcst.gen_pm()
    fcst.gen_fisher(mu_integral_step, skipgen=False)
    print("Relic Forecast ", fidx, " complete...")
    
for fidx, fcst in enumerate(neutrinoforecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(mu_integral_step, skipgen=False)
    print("Neutrino Forecast ", fidx, " complete...")

In [ ]:
#inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/CMBS4_Fisher_Relic_"
relic_outpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/relic/relicneutrinomatch/omega_ncdm/"

for fidx, fval in enumerate(relicforecastset):
    #relicforecastset[fidx].load_cmb_fisher(inpath+str(fidx+1)+".dat")
    relicforecastset[fidx].load_cmb_fisher(fisherpath="/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/CMBS4_Fisher_Neutrinos.dat")
    relicforecastset[fidx].export_matrices()
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_cmbfisher.mat", relic_outpath+str(fidx+1)+"/inv_cmbfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_lssfisher.mat", relic_outpath+str(fidx+1)+"/inv_lssfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_fullfisher.mat", relic_outpath+str(fidx+1)+"/inv_fullfisher.mat")
    
neutrino_inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/CMBS4_Fisher_Neutrinos.dat"
neutrino_outpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/neutrino/relicneutrinomatch/omega_ncdm/"

for fidx, fval in enumerate(neutrinoforecastset):
    neutrinoforecastset[fidx].load_cmb_fisher(neutrino_inpath)
    neutrinoforecastset[fidx].export_matrices()
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_cmbfisher.mat", neutrino_outpath+str(fidx+1)+"/inv_cmbfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_lssfisher.mat", neutrino_outpath+str(fidx+1)+"/inv_lssfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_fullfisher.mat", neutrino_outpath+str(fidx+1)+"/inv_fullfisher.mat")

In [ ]:
relicerromegancdmlss = [np.sqrt(fval.fisher[6,6]) for fidx, fval in enumerate(relicforecastset)]
neutrinoerromegancdmlss = [np.sqrt(fval.fisher[6,6]) for fidx, fval in enumerate(neutrinoforecastset)]

In [ ]:
masses = np.geomspace(0.1, 10.0, 21) 

plt.figure(figsize=(15,7.5))
plt.semilogx(masses, relicerromegancdmfull, label="CMB+LSS")
plt.semilogx(masses, relicerromegancdmlss, label="LSS")
plt.semilogx(masses, relicerromegancdmcmb, label="CMB")
plt.title("Relic Mass Uncertainty")
plt.xlabel("Equivalent Neutrino Mass [eV]")
plt.ylabel(r"$\sigma_M$")
plt.legend()
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(masses, neutrinoerrMncdmfull, label="CMB+LSS")
plt.semilogx(masses, neutrinoerrMncdmlss, label="LSS")
plt.semilogx(masses, neutrinoerrMncdmcmb, label="CMB")
plt.title("Neutrino Mass Uncertainty")
plt.xlabel("Sum of Neutrino Masses [eV]")
plt.ylabel(r"$\sigma_M$")
plt.legend()
plt.show()

# Load Pre-Existing Results

In [ ]:
inpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/relic/neff_reduced_for_neutrino_forecast_comparison/"

relicmasses = np.geomspace(0.1, 10.0, 21) * np.power(3., 1./4.)

relic_lss_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_lssfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(relicmasses)]
relic_cmb_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_cmbfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(relicmasses)]
relic_full_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_fullfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(relicmasses)]

relicerromegancdmlss = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(relic_lss_fishers)]
relicerromegancdmcmb = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(relic_cmb_fishers)]
relicerromegancdm = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(relic_full_fishers)]

In [ ]:
inpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/neutrino/"

neutrinomasses = np.geomspace(0.1, 10.0, 21) * (1./3.)

neutrino_lss_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_lssfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(neutrinomasses)]
neutrino_cmb_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_cmbfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(neutrinomasses)]
neutrino_full_fishers = [pd.read_csv(inpath+str(midx+1)+"/inv_fullfisher.mat", sep='\t', header=None, skiprows=1) 
    for midx, mval in enumerate(neutrinomasses)]

neutrinoerrmncdmlss = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(neutrino_lss_fishers)]
neutrinoerrmncdmcmb = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(neutrino_cmb_fishers)]
neutrinoerrmncdm = [np.sqrt(fval.iloc[6,6]) for fidx, fval in enumerate(neutrino_full_fishers)]

neutrinoerromegancdmlss = 3. * np.array(neutrinoerrmncdmlss) / 94.
neutrinoerromegancdmcmb = 3.* np.array(neutrinoerrmncdmcmb) / 94.
neutrinoerromegancdm = 3. * np.array(neutrinoerrmncdm) / 94.

# Other Diagnostics

### Print power spectra and related values at benchmark k, mu

In [ ]:
neutrinoforecast.print_v_table(k_index=92)
neutrinoforecast.print_P_table(k_index=92, mu_index=20)

In [ ]:
relicforecast.print_v_table(k_index=92)
relicforecast.print_P_table(k_index=92, mu_index=20)

### Compare forecast inferred Pm to CLASS inferred Pm

In [ ]:
#For each z, plot inferred spectrum vs. CLASS data
sns.set()
sns.set_palette("Blues_d", n_colors=2*len(z_table)+1)
plt.figure(figsize=(15, 7.5))
for zidx, zval in enumerate(z_table[0:-1:3]): 
    ps = neutrinoforecast.spectra_mid[zidx]
    plt.semilogx(ps.k_table, neutrinoforecast.Pm[zidx], label=("z = " + str(zval)))
    plt.semilogx(ps.class_pk['k (h/Mpc)']*fid['h'],
             np.array(ps.class_pk['P (Mpc/h)^3'] / np.power(neutrinofid['h'],3.)),
             label='CLASS P(k) Data',
             marker='x',
             linestyle=':')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_m$ [Mpc$^3$]')
plt.legend()
plt.show()
#Also plot difference between the two 

In [ ]:
#For each z, plot inferred spectrum vs. CLASS data
sns.set()
sns.set_palette("Blues_d", n_colors=2*len(z_table)+1)
plt.figure(figsize=(15, 7.5))
for zidx, zval in enumerate(z_table[0:-1:3]): 
    ps = relicforecast.spectra_mid[zidx]
    plt.semilogx(ps.k_table, relicforecast.Pm[zidx], label=("z = " + str(zval)))
    plt.semilogx(ps.class_pk['k (h/Mpc)']*relicfid['h'],
             np.array(ps.class_pk['P (Mpc/h)^3'] / np.power(relicfid['h'],3.)),
             label='CLASS P(k) Data',
             marker='x',
             linestyle=':')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_m$ [Mpc$^3$]')
plt.legend()
plt.show()
#Also plot difference between the two 

### Confirm transfer functions match

In [ ]:
zidx=0

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].b_interp_table-relicforecast.spectra_mid[zidx].b_interp_table, 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].b_interp_table-relicforecast2.spectra_mid[zidx].b_interp_table, 
             label="Relic m_ncdm Fixed")
#plt.semilogx(np.array(relicforecast.spectra_mid[zidx].k_table), 
#             relicforecast.spectra_mid[zidx].b_interp_table, 
#             label="Relic")
plt.title(r'Baryonic Matter Transfer Function, $\Sigma m_\nu = 0.06 eV, z=0.65$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta d_b$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].cdm_interp_table-relicforecast.spectra_mid[zidx].cdm_interp_table, 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].cdm_interp_table-relicforecast2.spectra_mid[zidx].cdm_interp_table, 
             label="Relic m_ncdm Fixed")
#plt.semilogx(np.array(relicforecast.spectra_mid[zidx].k_table), 
#             relicforecast.spectra_mid[zidx].cdm_interp_table, 
#             label="Relic")
plt.title(r'Cold Dark Matter Transfer Function, $\Sigma m_\nu = 0.06 eV, z=0.65$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta d_{cdm}$')
plt.legend()
plt.show()

### Confirm growth factors match

In [ ]:
for zidx, zval in enumerate(z_table):
    print(neutrinoforecast.spectra_mid[0].D, ", ", relicforecast2.spectra_mid[0].D)

### Confirm P_m match

In [ ]:
zidx = 0

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].ps_table-relicforecast.spectra_mid[zidx].ps_table, 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             neutrinoforecast.spectra_mid[zidx].ps_table-relicforecast2.spectra_mid[zidx].ps_table, 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in Matter Power Spectrum')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta P_m$')
plt.legend()
plt.show()

### Confirm P_g match

In [ ]:
zidx = 10
muidx = 10

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.Pg)[zidx, :, muidx] - np.array(relicforecast.Pg)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.Pg)[zidx, :, muidx] - np.array(relicforecast2.Pg)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in Galaxy Power Spectrum')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta P_g$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.Pm)[zidx, :] - np.array(relicforecast.Pm)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.Pm)[zidx, :] - np.array(relicforecast2.Pm)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in Matter Power Spectrum')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta P_m$')
plt.legend()
plt.show()

###  Confirm RSD, FOG, AP, COV match

In [ ]:
zidx = 10
muidx = 1

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.RSD)[zidx, :, muidx] - np.array(relicforecast.RSD)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.RSD)[zidx, :, muidx] - np.array(relicforecast2.RSD)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in RSD Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta RSD$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.FOG)[zidx, :, muidx] - np.array(relicforecast.FOG)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.FOG)[zidx, :, muidx] - np.array(relicforecast2.FOG)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in FOG Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta FOG$')
plt.legend()
plt.show()


### Confirm omega_ncdm derivatives match

In [ ]:
zidx=10

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogRSDdomega_ncdm)[zidx, :] - np.array(relicforecast.dlogRSDdomega_ncdm)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogRSDdomega_ncdm)[zidx, :] - np.array(relicforecast2.dlogRSDdomega_ncdm)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogRSD/domega_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogRSD/domega\_ncdm$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogFOGdomega_ncdm)[zidx, :] - np.array(relicforecast.dlogFOGdomega_ncdm)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogFOGdomega_ncdm)[zidx, :] - np.array(relicforecast2.dlogFOGdomega_ncdm)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogFOG/domega_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogFOG/domega\_ncdm$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogAPdomega_ncdm)[zidx, :] - np.array(relicforecast.dlogAPdomega_ncdm)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogAPdomega_ncdm)[zidx, :] - np.array(relicforecast2.dlogAPdomega_ncdm)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogAP/domega_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogAP/domega\_ncdm$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogCOVdomega_ncdm)[zidx, :] - np.array(relicforecast.dlogCOVdomega_ncdm)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogCOVdomega_ncdm)[zidx, :] - np.array(relicforecast2.dlogCOVdomega_ncdm)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogCOV/domega_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogCOV/domega\_ncdm$')
plt.legend()
plt.show()

### Confirm P_m derivatives match

In [ ]:
zidx = 12
muidx = 5

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdA_s)[zidx, :] - np.array(relicforecast.dlogPmdA_s)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdA_s)[zidx, :] - np.array(relicforecast2.dlogPmdA_s)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPm/dA_s Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/dA_s')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdn_s)[zidx, :] - np.array(relicforecast.dlogPmdn_s)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdn_s)[zidx, :] - np.array(relicforecast2.dlogPmdn_s)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPm/dn_s Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/dn_s')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdomega_b)[zidx, :] - np.array(relicforecast.dlogPmdomega_b)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdomega_b)[zidx, :] - np.array(relicforecast2.dlogPmdomega_b)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPm/domega_b Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/domega_b')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdomega_cdm)[zidx, :] - np.array(relicforecast.dlogPmdomega_cdm)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdomega_cdm)[zidx, :] - np.array(relicforecast2.dlogPmdomega_cdm)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPm/domega_cdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/domega_cdm')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdh)[zidx, :] - np.array(relicforecast.dlogPmdh)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdh)[zidx, :] - np.array(relicforecast2.dlogPmdh)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPm/dh Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/dh')
plt.legend()
plt.show()


sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdtau_reio)[zidx, :] - np.array(relicforecast.dlogPmdtau_reio)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPmdtau_reio)[zidx, :] - np.array(relicforecast2.dlogPmdtau_reio)[zidx, :], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPm/dtau_reio Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/dtau_reio')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             (np.array(neutrinoforecast.dlogPmdomega_ncdm)[zidx, :] - np.array(relicforecast.dlogPmdomega_ncdm)[zidx, :]), 
             label="Relic T_ncdm Fixed")
#plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
#             np.array(neutrinoforecast.dlogPmdomega_ncdm)[zidx, :] - np.array(relicforecast2.dlogPmdomega_ncdm)[zidx, :], 
#             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPm/domega_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/domega_ncdm')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             (np.array(neutrinoforecast.dlogPmdM_ncdm)[zidx, :] - np.array(relicforecast.dlogPmdM_ncdm)[zidx, :]), 
             label="Relic T_ncdm Fixed")
plt.title(r'Difference in dlogPm/dM_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPm/dM_ncdm')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(relicforecast.dlogPmdM_ncdm)[zidx, :], 
             label="Relic T_ncdm Fixed")
plt.title(r'dlogPm/dM_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'dlogPm/dM_ncdm')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(relicforecast2.dlogPmdT_ncdm)[zidx, :], 
             label="Relic M_ncdm Fixed")
plt.title(r'dlogPm/dT_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'dlogPm/dT_ncdm')
plt.legend()
plt.show()


### Confirm P_g derivatives match

In [ ]:
zidx = 10
muidx = 17

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdA_s)[zidx, :, muidx] - np.array(relicforecast.dlogPgdA_s)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdA_s)[zidx, :, muidx] - np.array(relicforecast2.dlogPgdA_s)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPg/dA_s Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogPg/dA_s$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdn_s)[zidx, :, muidx] - np.array(relicforecast.dlogPgdn_s)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdn_s)[zidx, :, muidx] - np.array(relicforecast2.dlogPgdn_s)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPg/dn_s Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogPg/dn_s$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdomega_b)[zidx, :, muidx] - np.array(relicforecast.dlogPgdomega_b)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdomega_b)[zidx, :, muidx] - np.array(relicforecast2.dlogPgdomega_b)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPg/domega_b Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogPg/domega_b$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdomega_cdm)[zidx, :, muidx] - np.array(relicforecast.dlogPgdomega_cdm)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdomega_cdm)[zidx, :, muidx] - np.array(relicforecast2.dlogPgdomega_cdm)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPg/domega_cdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogPg/domega_cdm$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdh)[zidx, :, muidx] - np.array(relicforecast.dlogPgdh)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdh)[zidx, :, muidx] - np.array(relicforecast2.dlogPgdh)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPg/dh Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogPg/dh$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdtau_reio)[zidx, :, muidx] - np.array(relicforecast.dlogPgdtau_reio)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdtau_reio)[zidx, :, muidx] - np.array(relicforecast2.dlogPgdtau_reio)[zidx, :, muidx], 
             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPg/dtau_reio Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogPg/dtau_reio$')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             (np.array(neutrinoforecast.dlogPgdomega_ncdm)[zidx, :, muidx] - np.array(relicforecast.dlogPgdomega_ncdm)[zidx, :, muidx]), 
             label="Relic T_ncdm Fixed")
#plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
#             np.array(neutrinoforecast.dlogPgdomega_ncdm)[zidx, :, muidx] - np.array(relicforecast2.dlogPgdomega_ncdm)[zidx, :, muidx], 
#             label="Relic m_ncdm Fixed")
plt.title(r'Difference in dlogPg/domega_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta dlogPg/domega_ncdm$')
plt.legend()
plt.show()


sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(neutrinoforecast.dlogPgdM_ncdm)[zidx, :, muidx] - np.array(relicforecast.dlogPgdM_ncdm)[zidx, :, muidx], 
             label="Relic T_ncdm Fixed")
plt.title(r'Difference in dlogPg/dM_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$\Delta$ dlogPg/dM_ncdm')
plt.legend()
plt.show()

sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(neutrinoforecast.spectra_mid[zidx].k_table), 
             np.array(relicforecast2.dlogPgdT_ncdm)[zidx, :, muidx], 
             label="Relic M_ncdm Fixed")
plt.title(r'dlogPg/dT_ncdm Term')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'dlogPg/dM_ncdm')
plt.legend()
plt.show()


# Light Relic Science Plot

We are now interested in studying how the uncertainty in relic temperature varies with the relic mass. 

### Setup

In [ ]:
#Imports 
import cosmicfish as cf 
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

#Plotting setup
sns.set()

#Workspace setup
projectdir = cf.correct_path("~/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/datastore/")
classpath = os.path.join(projectdir, "class")

#Analysis setup
derivative_step = 0.005 
mu_integral_step = 0.01 

#Experiment setup
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. 

#Fiducial cosmology  
fiducial = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : 1.5/2.726, # Units [T_cmb]  Are Julian's matrices at 1.64 K? 
        "m_ncdm" : 0.1, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, 
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

### Convergence Testing

In [ ]:
#Convergence testing 
convergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'relic', # 'relic' or 'neutrino' forecasting scheme 
    fiducial, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                          
            #'M_ncdm',                                    
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    mu_step=mu_integral_step,
    varyfactors=[0.003, 0.005, 0.007], # Relative factors used to compute convergence
    saveplots=True, 
    showplots=True,
    savepath="/Users/nicholasdeporzio/Desktop/",
    plotparams=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                          
            'omega_ncdm',                                    
            'T_ncdm',                                 
            'sigmafog',                                   
            'b0',                                         
            'alpha_k2']
    )

In [ ]:
convergencetest.gen_all_plots() # Display convergence plots

### Forecasting

In [ ]:
masses = np.geomspace(0.1, 10.0, 21) 
fiducialset = [dict(fiducial, **{'m_ncdm': masses[midx]}) for midx, mval in enumerate(masses)]

forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fiducialset)]

for fidx, fcst in enumerate(forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                          
            #'M_ncdm',                                    
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
#Save all the LSS/CMB/Full fisher matrices  
for fidx, fcst in enumerate(forecastset):
    name = '/Users/nicholasdeporzio/Desktop/cfworkspace/results/relic/scienceresults_temp_1.50_K/fishers/full/FULL_Fisher_' + str(fidx+1) + '.dat'
    head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t b_0 \t alpha_k2'
    np.savetxt(name, fcst.numpy_full_fisher, delimiter='\t', header=head)

In [ ]:
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/CMBS4_Fisher_Relic_"
outpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/relic/scienceresults/"

for fidx, fval in enumerate(forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+".dat")
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    fval.export_matrices("~/Desktop")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_cmbfisher.mat", outpath+str(fidx+1)+"/inv_cmbfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_lssfisher.mat", outpath+str(fidx+1)+"/inv_lssfisher.mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_fullfisher.mat", outpath+str(fidx+1)+"/inv_fullfisher.mat")

In [ ]:
errors_Tncdm_cmb = [np.sqrt(fval.numpy_cmb_covariance[6,6]) for fidx, fval in enumerate(forecastset)]    
errors_Tncdm_lss = [np.sqrt(fval.numpy_lss_covariance[6,6]) for fidx, fval in enumerate(forecastset)]
errors_Tncdm_full = [np.sqrt(fval.numpy_full_covariance[6,6]) for fidx, fval in enumerate(forecastset)]

plt.figure(figsize=(15,7.5))
plt.semilogx(masses, errors_Tncdm_full, label="CMB+LSS")
#plt.semilogx(masses, errors_Tncdm_lss, label="LSS")
#plt.semilogx(masses, errors_Tncdm_cmb, label="CMB")
plt.title("Relic Temperature Uncertainty")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\sigma_T ~[K]$")
plt.legend()
plt.savefig("/Users/nicholasdeporzio/Desktop/results.png")
plt.show()